# Multipage applications

Official information about creating multi-page applications in dash can be found <a href="https://dash.plotly.com/urls">here</a>. In this page I just want to write down some of my experiments related to this feature.

## `dash.page_registry`

`dash.page_registry` is a special dictionary that contains information about the pages in the current application. So in the following example an application with two pages has been created. `Page with button` has a special button that stores `dash.page_registry`.

In [1]:
%%writefile multipage_applications_files/app.py
import dash
from dash import Dash, html, dcc

app = Dash(__name__, use_pages=True)

app.layout = html.Div([
    html.H1('Multi-page app with Dash Pages'),
    html.Div([
        html.Div(
            dcc.Link(f"{page['name']} - {page['path']}", href=page["relative_path"])
        ) for page in dash.page_registry.values()
    ]),
    dash.page_container
])

if __name__ == '__main__':
    app.run(debug=True)

Overwriting multipage_applications_files/app.py


In [2]:
%%writefile multipage_applications_files/pages/some_page.py
import dash
from dash import html, callback, Input, Output

import pickle

dash.register_page(__name__)
layout = html.Div([
    html.H1('Page with button'),
    html.Button(
        "Save page_registry",
        id="save-button",
    ),
    html.Div(id='dummy')
])

@callback(
    Output("dummy", "children"),
    Input("save-button", "n_clicks")
)
def save_callback(n_clicks):
    with open("page_registry", "wb") as f:
        pickle.dump(dash.page_registry, f)

Overwriting multipage_applications_files/pages/some_page.py


In [3]:
%%writefile multipage_applications_files/pages/home.py
import dash
from dash import html

dash.register_page(__name__, path="/")
layout = html.Div([
    html.H1('Basic home page')
])

Overwriting multipage_applications_files/pages/home.py


By running the following cell, you can run the dash application described in the previous three cells.

In [4]:
%%bash
cd multipage_applications_files
python3 app.py

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app 'app'
 * Debug mode: on
Error while terminating subprocess (pid=19264): 


Here you can explore the layers of the `dash.page_registry`.

In [6]:
import pickle
from IPython.display import JSON, HTML

with open("multipage_applications_files/page_registry", "rb") as f:
    j = pickle.load(f)

dict(j)

{'pages.home': {'module': 'pages.home',
  'supplied_path': '/',
  'path_template': None,
  'path': '/',
  'supplied_name': None,
  'name': 'Home',
  'supplied_title': None,
  'title': 'Home',
  'description': '',
  'order': 0,
  'supplied_order': None,
  'supplied_layout': None,
  'supplied_image': None,
  'image': None,
  'image_url': None,
  'redirect_from': None,
  'relative_path': '/',
  'layout': Div([H1('Basic home page')])},
 'pages.some_page': {'module': 'pages.some_page',
  'supplied_path': None,
  'path_template': None,
  'path': '/some-page',
  'supplied_name': None,
  'name': 'Some page',
  'supplied_title': None,
  'title': 'Some page',
  'description': '',
  'order': None,
  'supplied_order': None,
  'supplied_layout': None,
  'supplied_image': None,
  'image': None,
  'image_url': None,
  'redirect_from': None,
  'relative_path': '/some-page',
  'layout': Div([H1('Page with button'), Button(children='Save page_registry', id='save-button'), Div(id='dummy')])}}